In [1]:
import re,requests
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

from os import listdir
from os.path import isfile, join
from tqdm import tqdm

s = requests.Session()

In [2]:
def getTopicsUrl():
    topic = {}
    base_url = 'http://www.krasotaimedicina.ru/diseases/'
    page = s.get(base_url)
    soup = BeautifulSoup(page.content, 'lxml')
    
    lis = soup.find_all("ul", {"class":"left-menu"})[0].find_all('li')
    
    for li in lis:
        name = li.text
        link = 'http://www.krasotaimedicina.ru' + str(li.find('a')['href'])
        topic[name] = link
    
    return topic

In [3]:
def getTopicDisease(furl):
    allDiseases = []
    url = furl
    while True:
        page = s.get(url)
        soup = BeautifulSoup(page.content, 'lxml')
        mu = soup.find_all("div", {"class":"element elementCosEqPr disLink"})
        
        for i in mu:
            a = i.find('a', {'class': 'title'})
            allDiseases.append(a.text)
        
        nextLink = soup.find_all('a', {'id': 'navigation_1_next_page'})
        if(len(nextLink) == 0):
            break
        
        url = 'http://www.krasotaimedicina.ru' + str(nextLink[0]['href'])
        
    return allDiseases

In [4]:
topics = getTopicsUrl()

In [5]:
len(topics)

31

In [6]:
tmp = getTopicDisease('http://www.krasotaimedicina.ru/diseases/speech-disorder')

In [7]:
len(tmp)

33

In [8]:
topic_diseases = {}
for topic in tqdm(topics.keys()):
    diseases = getTopicDisease(topics[topic])
    topic_diseases[topic] = diseases
    
    if(len(diseases) < 10):
        print(topic)

100%|██████████| 31/31 [04:26<00:00,  4.25s/it]


In [9]:
len(topic_diseases)

31

In [10]:
cnt = 0
for topic in topic_diseases.keys():
    cnt += len(topic_diseases[topic])

In [11]:
cnt

4099

In [12]:
bolezni = pd.read_csv('bolezni.csv')

In [13]:
all_bolezni = set(bolezni['name'].values)

In [16]:
top_column = []
for i in bolezni['name']:
    tmp_ar = []
    for key in topic_diseases.keys():
        tmp_s = set(topic_diseases[key])
        if(i in tmp_s):
            tmp_ar.append(key)
    top_column.append(' $ '.join(tmp_ar))

In [18]:
bolezni['topics'] = top_column

In [19]:
bolezni.head()

,name,descr,topics
0,Hallux valgus,Hallux valgus вальгусная деформация первого па...,Болезни ОДС и травмы
1,HELLP-синдром,Хотя последние годы HELLP синдром наблюдается ...,Женские болезни
2,Абдоминальная мигрень,Термин абдоминальная мигрень используется невр...,Нервные болезни
3,Абиотрофия сетчатки,Абиотрофия сетчатки дегенерация сетчатки дистр...,Глазные болезни $ Наследственные болезни
4,Абсанс,Первые упоминания абсанс датируются годом .Тер...,Нервные болезни


In [21]:
# bolezni.to_csv('bolezni.csv', index=False)